In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

import numpy as np

import tqdm

cuda_available = torch.cuda.is_available()

device = torch.device('cuda' if cuda_available else 'cpu')

pin_memory = True if cuda_available else False
batch_size = 128
latent_size = 1024
lr = 0.0003


#print(np.mean(train_dataset.data.numpy())/255)
#print(np.std(train_dataset.data.numpy())/255)


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=0.5, std=0.5)
    ])

train_dataset = datasets.MNIST(root='./data/mnist/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data/mnist/', train=False, transform=transform, download=False)

# train_dataset.data = train_dataset.data[:1000]


train = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.seq = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=3, stride=2, padding=0, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=3, stride=2, padding=1, bias=True),
            nn.Tanh()
            )
        
        
    def forward(self, x):
        x = self.seq(x)
        x = x[:, :, 0:28, 0:28]
        return x
        
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.seq = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),
            nn.Conv2d(512, 1,  kernel_size=3, stride=2, padding=1, bias=True),
            nn.Sigmoid()
            )
        
        
    def forward(self, x):
        x = self.seq(x)
        return x
        
G = Generator().to(device)

D = Discriminator().to(device)

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1:
        nn.init.kaiming_normal_(m.weight.data, nonlinearity='leaky_relu')
    elif classname.find('ConvTranspose2d') != -1:
        nn.init.kaiming_normal_(m.weight.data, nonlinearity='leaky_relu')
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
        
G = G.apply(weights_init)
D = D.apply(weights_init)

loss = nn.BCELoss()

G_opt = optim.AdamW(G.parameters(), lr=lr, betas=(0.5, 0.999))
D_opt = optim.AdamW(D.parameters(), lr=lr, betas=(0.5, 0.999))

def train_epoch(G_opt, D_opt, G, D, train):
    
    real_label = torch.tensor([1.0], device=device)
    fake_label = torch.tensor([0.0], device=device)
    
    G.train()
    D.train()
    
    dl = []
    gl = []
    
    for i, (x, _) in enumerate(tqdm.tqdm(train,0)):
        
        x_real = x.to(device)
        
        D_opt.zero_grad(set_to_none=True)
        
        D_out_real = D(x_real).view(-1)
        
        y_real = real_label.repeat(D_out_real.shape[0],)
        y_fake = fake_label.repeat(D_out_real.shape[0],)
        
        latent = torch.randn(D_out_real.shape[0], latent_size, 1, 1, device=device)
        
        
        with torch.no_grad():
            x_fake = G(latent)
        
        D_out_fake = D(x_fake).view(-1)
        
        D_real_loss = loss(D_out_real, y_real)
        D_fake_loss = loss(D_out_fake, y_fake)
        
        D_loss = D_real_loss + D_fake_loss
        D_loss.backward()
        D_opt.step()
        
        G_opt.zero_grad(set_to_none=True)
        
        x_fake = G(latent)
        D_out = D(x_fake).view(-1)
        
        G_loss = loss(D_out, y_real)
        
        G_loss.backward()
        G_opt.step()
        
        dl.append(D_loss.item())
        gl.append(G_loss.item())
        
    return np.mean(dl), np.mean(gl)



In [4]:
# retrain = True
retrain = False
epochs = 100

if retrain:
    for epoch in range(epochs):
        dl, gl = train_epoch(G_opt, D_opt, G, D, train)
        print(f"Epoch: {epoch}, D_loss: {dl}, G_loss: {gl}")
        torch.save(G.state_dict(), f"./gancheckpoints/generator_{epoch}.pth")
        torch.save(D.state_dict(), f"./gancheckpoints/discriminator_{epoch}.pth")
        with torch.no_grad():
            test_z = Variable(torch.randn(batch_size, 1024, 1, 1).to(device))
            generated = G(test_z)
            save_image(generated, './samples/sample_' + str(epoch) + '.png')



else:
    G.load_state_dict(torch.load(f"./gancheckpoints/generator_{81}.pth"))
    D.load_state_dict(torch.load(f"./gancheckpoints/discriminator_{81}.pth"))





In [ ]:
#create a convnet to find the latent space representation of each image
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
#now we can generate new training data
images = train_dataset.data
for i in range(0,100):
    #select 2 images at random
    image1 = images[np.random.randint(0,images.shape[0])]
    image2 = images[np.random.randint(0,images.shape[0])]

    #find the latent space representation of each image
    
        

100%|██████████| 8/8 [00:03<00:00,  2.08it/s]


6.707550883293152   0.0068160719238221645


 50%|█████     | 4/8 [00:02<00:02,  1.62it/s]


KeyboardInterrupt: 